In [69]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

% matplotlib inline

In [70]:
def defineAnnuli(r,nAnn):
    return np.append(0,np.sort(r)[np.linspace(0,len(r)-1,nAnn).astype(int)][1:])

def labelGalaxies(data,annuli):
    #adds a column to data indicating the galaxy's membership in an annulus
    bla = np.zeros(len(data['r']))
    for i in range(len(annuli)-1):
        bla += i*np.array(np.logical_and(data['r']>annuli[i],data['r']<=annuli[i+1]).astype(int))
    return bla

def galInAnn(data,annuli):
    #calculates the number of galaxies in all the annuli
    return np.array([(data['annuli']==i).sum() for i in range(len(annuli)-1)])

def sigInAnn(data,annuli):
    return [np.std(data['vobs'][data['annuli']==i]) for i in range(len(annuli)-1)]



In [71]:
data=pd.read_csv('mock_cluster.csv')
data.rename(columns={'Unnamed: 0':'id','0':'origAnn','1':'x','2':'y','3':'z','4':'vobs'},inplace=True)

data['r'] = np.power(np.power(data['x'],2.)+np.power(data['y'],2.),0.5)

nAnn = 10 #number of desired annuli
annuli = defineAnnuli(data['r'],nAnn) 

#summarize the true data

# number of galaxies per annulus
    
data['annuli']=labelGalaxies(data,annuli)
N = galInAnn(data,annuli)
sigma = sigInAnn(data,annuli)

print N
print sigma

[111 110 111 110 110 111 110 110 111]
[1101.230711133204, 978.8851193388302, 892.4168507756079, 670.3298668814153, 640.2207695327382, 598.3845099834416, 551.6151892055345, 504.1988352731031, 418.54940143523453]


In [72]:
def nuProfile(Ntot,radii,k=-3.):
    nu0 = Ntot*k/((1+radii[-1])**(k+1)-1)
    return (nu0*((1+radii[1:])**(k+1)-(1+radii[:-1])**(k+1))/k).astype(int)

def sigmaProfile(radii,sigMax=1000,k=-.2):
    return sigMax*(0.5*(radii[1:]+radii[:-1]))**k

In [123]:
def createMockData(radii,Ntot=1000,sigMax=1000,kindex=-3,sigmaindex=-0.2): #total Ngals, no of shells
    clusterData = np.zeros((2*Ntot,2))
    counter=0
    density = nuProfile(Ntot,radii,k=kindex)
    sig = sigmaProfile(radii,sigMax,k=sigmaindex)
    
    for shell,dens in enumerate(np.random.poisson(density)):
        #generate positions
        phi = np.random.uniform(0.,2*np.pi,dens)
        cosTheta = np.random.uniform(-1,1,dens)
        theta = np.arccos(cosTheta)
        u = np.random.uniform(radii[shell]**3,radii[shell+1]**3,dens) #cuberoot([rin,rout])
        r = np.power(u,1./3)
        
        x = r*np.sin(theta)*np.cos(phi)
        y = r*np.sin(theta)*np.sin(phi)
        z = r*cosTheta
        
        r = np.power(np.power(x,2.)+np.power(x,2.),0.5)
        #generate velocities
        
        speed = sig[shell]*np.random.randn(dens)
        #phi = np.random.uniform(0.,2*np.pi,dens)
        cosTheta = np.random.uniform(-1,1,dens)
        #theta = np.arccos(cosTheta)
        
        v=speed*cosTheta
        
        #replace with vstack,append
        clusterData[counter:counter+dens]=np.hstack((r.reshape((dens,1)),v.reshape((dens,1))))
        counter += dens
    
    print 'Created mock data with '+str(counter)+' galaxies.'
    return pd.DataFrame(clusterData[:counter],columns=['r','vobs'])

In [128]:
kIndex = -3
sigmaIndex = -0.2
sigMax = 1000

mockData = createMockData(annuli,len(data['r']),sigMax,kIndex,sigmaIndex)
mockData['annuli'] = labelGalaxies(mockData,annuli)
mockN = galInAnn(mockData,annuli)
mockSigma = sigInAnn(mockData,annuli)

print mockN
print mockSigma

Created mock data with 1035 galaxies.
[218 112 115 117 126  66  97  76 108]
[916.5182504241521, 695.8228627606603, 699.9997985885665, 783.0581612163897, 766.4275798727011, 697.3522563943602, 564.5563620259313, 632.9823675709375, 390.2067777227221]


Created mock data with 2069 galaxies.
Created mock data with 2014 galaxies.
Created mock data with 1918 galaxies.
Created mock data with 1928 galaxies.
Created mock data with 2016 galaxies.
Created mock data with 1987 galaxies.
100 loops, best of 3: 17.6 ms per loop
